In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoProcessor,
    GenerationConfig,
)
from PIL import Image


repo_name = "cyan2k/molmo-7B-D-bnb-4bit"
arguments = {"device_map": "auto", "torch_dtype": "auto", "trust_remote_code": True}


# load the processor
processor = AutoProcessor.from_pretrained(repo_name, **arguments)

# load the model
model = AutoModelForCausalLM.from_pretrained(repo_name, **arguments)


/home/dnth/mambaforge-pypy3/envs/cv-docker-images/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-30 14:44:54.403794: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 14:44:54.412425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730270694.423642   73721 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730270694.426992   73721

In [2]:
%%time
# load image and prompt
inputs = processor.process(
    images=[Image.open("sample_images/0a6ee446579d2885.jpg"), Image.open("sample_images/0a6ee446579d2885.jpg"), Image.open("sample_images/0a6ee446579d2885.jpg")],
    text="Describe this image.",
)


inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

# generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=20, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer,
)

# only get generated tokens; decode them to text
generated_tokens = output[0, inputs["input_ids"].size(1) :]
generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# print the generated text
print(generated_text)

W0000 00:00:1730270702.213889   73721 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


 The image depicts a woman, likely in her 40s or 50s, standing
CPU times: user 6.03 s, sys: 949 ms, total: 6.98 s
Wall time: 6.59 s


In [3]:
generated_text

' The image depicts a woman, likely in her 40s or 50s, standing'